In [ ]:
"""
Candidate Elimination Algorithm Demonstration
"""

import pandas as pd

class StepByStepCE:
    def __init__(self):
        self.S = None
        self.G = None
        self.attributes = None
        self.attribute_values = None

    def setup(self, attributes, attribute_values):
        """Setup the algorithm"""
        self.attributes = attributes
        self.attribute_values = attribute_values
        n_attr = len(attributes)

        # Initialize hypotheses
        self.S = ['0'] * n_attr  # Most specific
        self.G = [['?'] * n_attr]  # Most general

        print("CANDIDATE ELIMINATION ALGORITHM")
        print("="*50)
        print(f"Attributes: {attributes}")
        print(f" Initial Hypotheses:")
        print(f"S = {self.S}")
        print(f"G = {self.G}")
        return self

    def step(self, example, label, step_num):
        """Process one training example"""
        if label == 'positive':
            self._positive_step(example)
        else:
            self._negative_step(example)

        print(f" Step {step_num}: {example} -> {label}")
        print(f"S = {self.S}")
        print(f"G = {self.G}")

        if not self.G:
            print(" Version space collapsed!")

        return self

    def _positive_step(self, example):
        """Handle positive example"""
        # Generalize S
        for i in range(len(self.S)):
            if self.S[i] == '0':
                self.S[i] = example[i]
            elif self.S[i] != example[i]:
                self.S[i] = '?'

        # Filter G - remove hypotheses that don't cover positive example
        self.G = [g for g in self.G if self._covers(g, example)]

    def _negative_step(self, example):
        """Handle negative example"""
        # Check S consistency
        if self._covers(self.S, example):
            print(f"ERROR: S covers negative example - inconsistent data!")
            return

        # Specialize G
        new_G = []
        for g in self.G:
            if self._covers(g, example):
                # Create specializations
                for i in range(len(g)):
                    if g[i] == '?':
                        for val in self.attribute_values[i]:
                            if val != example[i]:
                                new_hyp = g.copy()
                                new_hyp[i] = val

                                if self._is_more_general_than_S(new_hyp):
                                    new_G.append(new_hyp)
            else:
                new_G.append(g)

        # Remove duplicates
        self.G = self._remove_duplicates(new_G)

    def _covers(self, hypothesis, instance):
        """Check if hypothesis covers instance"""
        for h_val, inst_val in zip(hypothesis, instance):
            if h_val != '?' and h_val != '0' and h_val != inst_val:
                return False
        return True

    def _is_more_general_than_S(self, hypothesis):
        """Check if hypothesis is more general than S"""
        for i in range(len(hypothesis)):
            if self.S[i] != '0' and self.S[i] != '?':
                if hypothesis[i] != '?' and hypothesis[i] != self.S[i]:
                    return False
        return True

    def _remove_duplicates(self, hypotheses):
        """Remove duplicate hypotheses"""
        unique = []
        for h in hypotheses:
            if h not in unique:
                unique.append(h)
        return unique

def main_demonstration():
    """Main demonstration with required outputs only"""

    # Car attributes
    attributes = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
    attribute_values = {
        0: ['vhigh', 'high', 'med', 'low'],
        1: ['vhigh', 'high', 'med', 'low'],
        2: ['2', '3', '4', '5more'],
        3: ['2', '4', 'more'],
        4: ['small', 'med', 'big'],
        5: ['low', 'med', 'high']
    }

    # Training examples with both positive and negative
    training_examples = [
        (['low', 'low', '4', 'more', 'big', 'high'], 'positive'),
        (['vhigh', 'vhigh', '2', '2', 'small', 'low'], 'negative'),
        (['med', 'med', '4', '4', 'med', 'high'], 'positive'),
        (['high', 'high', '2', '2', 'small', 'med'], 'negative'),
        (['low', 'med', '4', 'more', 'big', 'med'], 'positive'),
    ]

    print("Training Examples:")
    for i, (example, label) in enumerate(training_examples):
        print(f"  {i+1}: {example} -> {label}")

    # Run algorithm
    ce = StepByStepCE()
    ce.setup(attributes, attribute_values)

    for i, (example, label) in enumerate(training_examples):
        ce.step(example, label, i + 1)

    # Final results
    print(f" {'='*50}")
    print("FINAL RESULTS")
    print(f"{'='*50}")
    print(f"S = {ce.S}")
    print(f"G = {ce.G}")

    if ce.G:
        print(f" Version Space: Contains {len(ce.G)} general hypotheses + 1 specific hypothesis")
    else:
        print(f" Version Space: EMPTY (no consistent concept exists)")

    # Calculate accuracy
    print(f" {'='*30}")
    print("ACCURACY EVALUATION")
    print(f"{'='*30}")

    correct = 0
    total = len(training_examples)

    for i, (example, actual_label) in enumerate(training_examples):
        # Predict using S and G
        if not ce.G:  # Empty version space
            predicted = 'negative'
        else:
            s_covers = ce._covers(ce.S, example)
            g_covers = any(ce._covers(g, example) for g in ce.G)
            predicted = 'positive' if (s_covers and g_covers) else 'negative'

        is_correct = predicted == actual_label
        if is_correct:
            correct += 1

        print(f"Example {i+1}: {example}")
        print(f"  Actual: {actual_label}, Predicted: {predicted}")

    accuracy = correct / total
    print(f" Accuracy: {correct}/{total} = {accuracy:.3f}")

if __name__ == "__main__":
    main_demonstration()

Training Examples:
  1: ['low', 'low', '4', 'more', 'big', 'high'] -> positive
  2: ['vhigh', 'vhigh', '2', '2', 'small', 'low'] -> negative
  3: ['med', 'med', '4', '4', 'med', 'high'] -> positive
  4: ['high', 'high', '2', '2', 'small', 'med'] -> negative
  5: ['low', 'med', '4', 'more', 'big', 'med'] -> positive
CANDIDATE ELIMINATION ALGORITHM
Attributes: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
 Initial Hypotheses:
S = ['0', '0', '0', '0', '0', '0']
G = [['?', '?', '?', '?', '?', '?']]
 Step 1: ['low', 'low', '4', 'more', 'big', 'high'] -> positive
S = ['low', 'low', '4', 'more', 'big', 'high']
G = [['?', '?', '?', '?', '?', '?']]
 Step 2: ['vhigh', 'vhigh', '2', '2', 'small', 'low'] -> negative
S = ['low', 'low', '4', 'more', 'big', 'high']
G = [['low', '?', '?', '?', '?', '?'], ['?', 'low', '?', '?', '?', '?'], ['?', '?', '4', '?', '?', '?'], ['?', '?', '?', 'more', '?', '?'], ['?', '?', '?', '?', 'big', '?'], ['?', '?', '?', '?', '?', 'high']]
 Step 3: ['med